# AgentNeo: Code Assistant

In [ ]:
!pip install agentneo -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 914.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

# TODO: Save you OpenAI Key as a Secret
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## Authentication

In [ ]:
from agentneo import AgentNeo, Project, Dataset, Experiment, Tracer

In [ ]:
# To login using email-id, creates new account if is not a user else retrieves the existing account
BASE_URL = "http://74.249.60.46:5000"

agent_session = AgentNeo(email="test.user5@raga.ai", base_url=BASE_URL)

User not found for email: test.user5@raga.ai. Creating new user...
New user created for email: test.user5@raga.ai
Please keep your access key and secret key handy for future logins:
Access Key: YPpzZcI0TYa_8PYht2A77A
Secret Key: WJ_LO3mb9r3-Kga0MDRxEZiFXLWlslwZSwmHZ0oJ_v8


In [ ]:
access_key = "YPpzZcI0TYa_8PYht2A77A"
secret_key = "WJ_LO3mb9r3-Kga0MDRxEZiFXLWlslwZSwmHZ0oJ_v8"

# To login with the the access_key, secret_key
agent_session = AgentNeo(
        access_key=access_key,
        secret_key=secret_key,
        base_url=BASE_URL,
    )

## Project

In [ ]:
project_created = Project(session=agent_session, project_name="Project_77", description="A test project").create()
project_id = project_created['id']

Project 'Project_77' created successfully with ID: 36


## Tracing

In [ ]:
pip install langchain_openai langchain_community langgraph -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
#Initialise the tracer by providing the tools used and their description
tracer = Tracer(session=agent_session, metadata= {'tools': [
    {'name': 'structured_llm_response', 'description': "A language model that outputs structured responses based on the Code schema."},
    {'name': 'code_gen_prompt', 'description': "Prompt template to structure the code generation response using LangChain Expression Language."},
    {'name': 'check_llm_output', 'description': "Function to check for parsing errors or tool invocation failures in the LLM's output."},
]})

# Langgraph code
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import TypedDict, Annotated, Sequence
import operator
from langgraph.graph import StateGraph, END

# LLM configuration
openai_llm = ChatOpenAI(temperature=0.4,
                        model_name="gpt-4o-mini",
                        api_key=os.environ["OPENAI_API_KEY"],
                        callbacks=[tracer.get_callback_handler()])

# Prompt to enforce tool use
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """<instructions> You are a coding assistant with expertise in LCEL, LangChain expression language. \n
    Here is the LCEL documentation:  \n ------- \n  {context} \n ------- \n Answer the user question based on the \n
    above provided documentation. Ensure any code you provide can be executed with all required imports and variables \n
    defined. Structure your answer: 1) a prefix describing the code solution, 2) the imports, 3) the functioning code block. \n
    Invoke the code tool to structure the output correctly. </instructions> \n Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)

# Data model for structured output
class Code(BaseModel):
    """Code output"""
    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")
    description = "Schema for code solutions to questions about LCEL."

# Structured LLM with output parsing
structured_llm = openai_llm.with_structured_output(Code, include_raw=True)

# Optional: Check for errors in case tool use is flaky
def check_llm_output(tool_output):
    """Check for parse error or failure to call the tool"""
    if tool_output.get("parsing_error"):
        raw_output = tool_output["raw"]
        error = tool_output["parsing_error"]
        raise ValueError(
            f"Error parsing your output! Be sure to invoke the tool. Output: {raw_output}. \n Parse error: {error}"
        )
    elif not tool_output.get("parsed"):
        raise ValueError(
            "You did not use the provided tool! Be sure to invoke the tool to structure the output."
        )
    return tool_output

# Chain with output check
code_chain_raw = (
    code_gen_prompt | structured_llm | check_llm_output
)

def insert_errors(inputs):
    """Insert errors for tool parsing in the messages"""
    error = inputs["error"]
    messages = inputs["messages"]
    messages += [
        (
            "assistant",
            f"Retry. You are required to fix the parsing errors: {error} \n\n You must invoke the provided tool.",
        )
    ]
    return {
        "messages": messages,
        "context": inputs["context"],
    }

# This will be run as a fallback chain
fallback_chain = insert_errors | code_chain_raw
N = 3  # Max retries
code_gen_chain_re_try = code_chain_raw.with_fallbacks(
    fallbacks=[fallback_chain] * N, exception_key="error"
)

def parse_output(solution):
    """When we add 'include_raw=True' to structured output,
    it will return a dict w 'raw', 'parsed', 'parsing_error'."""
    return solution.get("parsed")

# Optional: With retry to correct for failure to invoke tool
code_gen_chain = code_gen_chain_re_try | parse_output

# No retry
code_gen_chain_no_retry = code_gen_prompt | structured_llm | parse_output

# Integration with Tracer and StateGraph

class AgentState(TypedDict):
    messages: Annotated[Sequence[str], operator.add]
    context: str

@tracer.trace_node
def agent(state):
    query = state["messages"]
    context = state["context"]
    prompt_value = code_gen_prompt.format(messages=query, context=context)
    res = openai_llm.invoke(prompt_value)
    # Accessing the actual content within the response object
    return {"messages": [res.content], "context": context}

@tracer.trace_node
def structured_llm_response(state):
    query = state["messages"]
    # Assuming query is a list of strings, you can directly pass it as a formatted prompt string
    prompt_value = code_gen_prompt.format(messages=query, context=state["context"])
    result = structured_llm.invoke(prompt_value)
    parsed_result = parse_output(result)
    return {"messages": [parsed_result.code], "context": state["context"]}

@tracer.trace_graph
def workflow():
    graph = StateGraph(AgentState)
    graph.add_node("agent", agent)
    graph.add_node("structured_llm_response", structured_llm_response)
    graph.add_edge("agent", "structured_llm_response")
    graph.set_entry_point("agent")
    graph.set_finish_point("structured_llm_response")
    return graph.compile()

compiled_workflow = workflow()

@tracer.trace_node
def get_output(message, context):
    inputs = {"messages": [message], "context": context}
    response = compiled_workflow.invoke(inputs)
    print(response["messages"][-1])

# Example usage for code assistance
# context = "You are working on a natural language processing project that requires the use of the LangChain Expression Language (LCEL) to dynamically build and evaluate logical expressions. LCEL allows you to create, manipulate, and evaluate expressions that can be embedded into larger workflows or applications. You have access to the LCEL documentation, which provides detailed information on its syntax, functions, and usage patterns.You are familiar with basic programming concepts and have some experience in Python, but LCEL is relatively new to you. You are looking to leverage LCEL to perform complex data manipulations and logical operations in your project."
# get_output('You are trying to create a logical expression in LCEL that checks if a list of strings contains any elements that match a specific pattern. For instance, you want to check if the list contains any strings that start with the prefix "LC_" and end with the suffix "_2024". How can you construct and evaluate this expression using LCEL? Please provide a code example that includes all necessary imports and defines the required variables.', context)
context ="You are participating in a data analysis project where you need to work with time series data. The dataset contains daily stock prices for multiple companies over several years. You are using the Pandas library in Python for data manipulation and analysis. You have experience with basic data operations in Pandas, such as filtering and aggregating data, but now you need to perform more complex time series analysis, including calculating moving averages and identifying trends."
get_output('You want to calculate a 30-day moving average of the stock prices and plot it along with the original daily prices for a specific company. How can you achieve this using Pandas? Provide a code example that includes all necessary imports and data manipulation steps.', context)

# Load the dataset
# For demonstration, let's assume you have a CSV file with stock prices
# The CSV file should have columns: 'Date', 'Company', 'Price'
data = pd.read_csv('stock_prices.csv', parse_dates=['Date'])

# Filter the data for a specific company
company_name = 'CompanyA'  # Replace with the desired company name
company_data = data[data['Company'] == company_name].set_index('Date')

# Calculate the 30-day moving average
company_data['30_day_moving_avg'] = company_data['Price'].rolling(window=30).mean()

# Plot the original prices and the moving average
plt.figure(figsize=(14, 7))
plt.plot(company_data.index, company_data['Price'], label='Daily Prices', color='blue')
plt.plot(company_data.index, company_data['30_day_moving_avg'], label='30-Day Moving Average', color='orange')
plt.title(f'{company_name} Stock Prices and 30-Day Moving Average')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()


In [ ]:
trace_id = tracer.upload_trace()

Trace uploaded successfully. Trace ID: 41


## Dataset

In [ ]:
# Create a dataset from a trace
dataset = Dataset(
    session=agent_session,
    project_id=project_id,
    dataset_name="Dataset_1",
    description="A test dataset"
)

dataset_traced = dataset.from_trace(trace_id=tracer.id, trace_filter=None)

## Experiment

In [ ]:
# Create Experiment
experiment = Experiment(
        session=agent_session,
        experiment_name="Experiment_3",
        description="A test experiment",
        dataset_id=dataset_traced['id'],
        project_id=project_id
    )

experiment_created = experiment.create()

In [ ]:
# Execute Experiment
exp = experiment.execute(metrics=[
    {"name": "summarise", "config": {}},
    {"name": "tool_selection_accuracy", "config": {"model": "gpt-4o-mini", "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]}},
    {"name": "tool_usage_efficiency", "config": {"model": "gpt-4o-mini", "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]}},
    {"name": "goal_decomposition_efficiency", "config": {"model": "gpt-4o-mini", "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]}},
    {"name": "plan_adaptibility", "config": {"model": "gpt-4o-mini", "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]}},
])

In [ ]:
# Experiment Results
exp = experiment.get_results(experiment_id=exp['id'])

for i in exp['results']:
    print(f"Name: {i['metric_name']}")
    print(f"Result:")
    for key, value in i['result'].items():
        print(f"{key}: {value}")
    print(f"{'*'*100}\n")

Name: summarise
Result:
completion_cost: 0.0028155
completion_tokens: 1877
input_cost: 0.001877
input_tokens: 3754
latency: 0.004678487777709961
summary: {'children': 'list with 1 items', 'end': 'NoneType', 'inputs': 'NoneType', 'name': 'str', 'outputs': 'NoneType', 'start': 'NoneType', 'type': 'str'}
total_cost: 0.0046925
total_tokens: 5631
****************************************************************************************************

Name: tool_selection_accuracy
Result:
reasoning: ["The selected tool, 'structured_llm_response', is highly appropriate for the task of generating a code example for calculating a 30-day moving average of stock prices using Pandas. The tool aligns perfectly with the user's request for structured output, and the AI's thought process demonstrates a clear understanding of the task requirements."]
score: 0.9999999900000002
selected_tools: ["structured_llm_response"]
****************************************************************************************